In [ ]:
import logging
import csv
import sqlite3
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler

# Set up logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)

# Database setup
def init_db():
    conn = sqlite3.connect('frs_data.db')
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS frs_scores (
            id INTEGER PRIMARY KEY,
            age INTEGER,
            sex TEXT,
            smoker BOOLEAN,
            total_cholesterol INTEGER,
            hdl_cholesterol INTEGER,
            systolic_bp INTEGER,
            treated_bp TEXT,
            risk_category TEXT
        )
    ''')
    conn.commit()
    return conn

# Save data to CSV
def save_to_csv(data):
    with open('frs_scores.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(data)

# Define risk categories based on FRS score
def risk_category(score):
    if score < 10:
        return "Low risk"
    elif 10 <= score < 20:
        return "Moderate risk"
    else:
        return "High risk"

# Calculate Framingham Risk Score
def calculate_frs(age, sex, smoker, total_cholesterol, hdl_cholesterol, systolic_bp, treated_bp):
    points = 0
    
    # Age points
    if age >= 30 and age <= 34:
        points += 0
    elif age <= 39:
        points += 2
    elif age <= 44:
        points += 5
    elif age <= 49:
        points += 6
    elif age <= 54:
        points += 8
    elif age <= 59:
        points += 10
    elif age <= 64:
        points += 11
    elif age <= 69:
        points += 12
    elif age <= 74:
        points += 14
    else:
        points += 15

    # Sex points (assuming male=1, female=0)
    if sex == 'Male':
        points += 1 * smoker * 4

    # Cholesterol points (simplified)
    if total_cholesterol < 200:
        points += 0
    elif total_cholesterol < 240:
        points += 1
    else:
        points += 2

    # HDL cholesterol points (simplified)
    if hdl_cholesterol < 40:
        points += 2
    elif hdl_cholesterol < 60:
        points += 0

    # Systolic BP and treatment (simplified)
    if treated_bp == 'Yes':
        if systolic_bp < 120:
            points -= 3
        elif systolic_bp < 130:
            points += 0
        else:
            points += (systolic_bp - 120) // 10

    return risk_category(points)

# Start command handler
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Welcome! I will help you calculate your Framingham Risk Score for heart disease.')
    
    # Initialize user data collection
    context.user_data.clear()   # Clear any previous data
    context.user_data['step'] = 'age'   # Start with the first question
    update.message.reply_text('Please enter your age (30-79 years):')

# Handle user input for FRS calculation step-by-step
def handle_message(update: Update, context: CallbackContext) -> None:
    user_data = context.user_data
    
    step = user_data.get('step')
    
    try:
        if step == 'age':
            user_data['age'] = int(update.message.text)
            user_data['step'] = 'sex'
            show_sex_buttons(update)
        
        elif step == 'smoker':
            user_data['smoker'] = update.message.text.capitalize() == 'Yes'
            user_data['step'] = 'total_cholesterol'
            update.message.reply_text('Enter your total cholesterol (mg/dL):')
        
        elif step == 'total_cholesterol':
            user_data['total_cholesterol'] = int(update.message.text)
            user_data['step'] = 'hdl_cholesterol'
            update.message.reply_text('Enter your HDL cholesterol (mg/dL):')
        
        elif step == 'hdl_cholesterol':
            user_data['hdl_cholesterol'] = int(update.message.text)
            user_data['step'] = 'systolic_bp'
            update.message.reply_text('Enter your systolic blood pressure (mmHg):')
        
        elif step == 'systolic_bp':
            user_data['systolic_bp'] = int(update.message.text)
            user_data['step'] = 'treated_bp'
            update.message.reply_text('Are you being treated for high blood pressure? (Yes/No):')
        
        elif step == 'treated_bp':
            user_data['treated_bp'] = update.message.text.capitalize()
            
            # Calculate FRS and respond to the user
            score_category = calculate_frs(
                user_data['age'], 
                user_data['sex'], 
                user_data['smoker'],
                user_data['total_cholesterol'], 
                user_data['hdl_cholesterol'],
                user_data['systolic_bp'], 
                user_data['treated_bp']
            )
            
            update.message.reply_text(f'Your Framingham Risk Score category is: {score_category}')
            
            # Save data to CSV and DB
            save_to_csv([
                user_data['age'],
                user_data['sex'],
                user_data['smoker'],
                user_data['total_cholesterol'],
                user_data['hdl_cholesterol'],
                user_data['systolic_bp'],
                user_data['treated_bp'],
                score_category
            ])
            
            save_to_db(user_data, score_category)

            # Clear user data for next calculation
            context.user_data.clear()

    except ValueError:
        update.message.reply_text("Please enter a valid number or response. Try again.")

def save_to_db(user_data, score_category):
    conn = init_db()
    cursor = conn.cursor()
    
    cursor.execute('''
        INSERT INTO frs_scores (age, sex, smoker, total_cholesterol, hdl_cholesterol, systolic_bp, treated_bp, risk_category) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        user_data['age'],
        user_data['sex'],
        int(user_data['smoker']),
        user_data['total_cholesterol'],
        user_data['hdl_cholesterol'],
        user_data['systolic_bp'],
        user_data['treated_bp'],
        score_category,
    ))
    
    conn.commit()
    conn.close()

def show_sex_buttons(update: Update) -> None:
    keyboard = [
        [InlineKeyboardButton("Male", callback_data='Male'),
         InlineKeyboardButton("Female", callback_data='Female')]
    ]
    
    reply_markup = InlineKeyboardMarkup(keyboard)
    
    update.message.reply_text('Select your sex:', reply_markup=reply_markup)

def show_smoker_buttons(update: Update) -> None:
    keyboard = [
        [InlineKeyboardButton("Yes", callback_data='Yes'),
         InlineKeyboardButton("No", callback_data='No')]
    ]
    
    reply_markup = InlineKeyboardMarkup(keyboard)
    
    update.message.reply_text('Are you a smoker?', reply_markup=reply_markup)

# Handle button callbacks for sex and smoking status selection
def button_handler(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    
    user_data = context.user_data
    
    step = user_data.get('step')
    
    if step == 'sex':
        user_data['sex'] = query.data
        user_data['step'] = 'smoker'
        show_smoker_buttons(query)
        
    elif step == 'smoker':
        user_data['smoker'] = query.data == 'Yes'
        user_data['step'] = 'total_cholesterol'
        query.edit_message_text(text="Enter your total cholesterol (mg/dL):")

# Main function to run the bot
def main() -> None:
    updater = Updater("****")

    updater.dispatcher.add_handler(CommandHandler("start", start))
    
    updater.dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_message))
    
    updater.dispatcher.add_handler(CallbackQueryHandler(button_handler))

    updater.start_polling()
    
    updater.idle()

if __name__ == '__main__':
   main()


2024-12-26 22:01:45,017 - apscheduler.scheduler - INFO - Scheduler started


In [1]:
4*5

20